In [25]:
import os
import numpy as np
from keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from PIL import Image, ImageChops, ImageEnhance
import cv2
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from sklearn.model_selection import train_test_split
from scipy import stats
from keras.models import Sequential
from keras.layers import BatchNormalization
from tensorflow.keras.applications import Xception
from sklearn.metrics import classification_report

In [31]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp.jpg'
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    ela_image = ImageChops.difference(image, temp_image)
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    return ela_image

In [30]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', activation = 'relu', input_shape = (224, 224, 3)))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation = 'relu',))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', activation = 'relu',))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
    return model

In [32]:
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 90).resize(image_size)).flatten() / 255.0

In [33]:
image_size = (224, 224)
X = []
y = []

In [34]:
# Define source paths
authentic = 'data/casia/au/'
tampered = 'data/casia/tp/'

In [35]:
for root, dirs, files in os.walk(authentic):
  for filename in files:
    file_path = os.path.join(root, filename)
    if filename.lower().endswith(('jpg', 'png')):
      X.append(prepare_image(file_path))
      y.append(1)
    if len(X)==3000:
        break

# random.shuffle(X)

print(len(X), len(y))

37 37


In [36]:
count = 0
for root, dirs, files in os.walk(tampered):
    for filename in files:
        file_path = os.path.join(root, filename)
        if filename.lower().endswith(('jpg', 'png', 'tif', 'tiff')):
            X.append(prepare_image(file_path))
            y.append(0)
        if len(X)==4500:
            break

In [37]:
X = np.array(X)
y = to_categorical(y, 2)
X = X.reshape(-1, 224, 224, 3)

In [38]:
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.2, random_state=5)
X = X.reshape(-1,1,1,1)
print(len(X_train), len(Y_train))
print(len(X_val), len(Y_val))

58 58
15 15


In [39]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5,mode='max', restore_best_weights=True)
early_stopping

In [40]:
model = build_model()
epochs = 30
batch_size = 32
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [41]:
hist = model.fit(
    X_train,
    Y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, Y_val),
    callbacks = [early_stopping])
hist

Epoch 1/30
2/2 [==============================] - 1s 440ms/step - loss: 4.5744 - accuracy: 0.6724 - val_loss: 0.5607 - val_accuracy: 0.6000
Epoch 2/30
2/2 [==============================] - 1s 340ms/step - loss: 10.7686 - accuracy: 0.7931 - val_loss: 0.5501 - val_accuracy: 0.8667
Epoch 3/30
2/2 [==============================] - 1s 333ms/step - loss: 6.1977 - accuracy: 0.9483 - val_loss: 0.8056 - val_accuracy: 0.9333
Epoch 4/30
2/2 [==============================] - 1s 299ms/step - loss: 4.3796 - accuracy: 0.9483 - val_loss: 1.6416 - val_accuracy: 0.4000
Epoch 5/30
2/2 [==============================] - 1s 297ms/step - loss: 3.6934 - accuracy: 0.9655 - val_loss: 1.1443 - val_accuracy: 0.4000
Epoch 6/30
2/2 [==============================] - 1s 301ms/step - loss: 0.9807 - accuracy: 0.9828 - val_loss: 0.8396 - val_accuracy: 0.6667
Epoch 7/30
2/2 [==============================] - 1s 305ms/step - loss: 0.3975 - accuracy: 0.9828 - val_loss: 1.0210 - val_accuracy: 0.4000
Epoch 8/30
2/2 [===

In [42]:
model.evaluate(X_val,Y_val)

1/1 [==============================] - 0s 55ms/step - loss: 0.8056 - accuracy: 0.9333


[0.8056190609931946, 0.9333333373069763]

In [46]:
y_test_labels = np.argmax(Y_val, axis=1)
y_pred_probs = model.predict(X_val)
y_pred_labels = np.argmax(y_pred_probs, axis=1)
print(classification_report(y_test_labels, y_pred_labels, target_names=['Class 0', 'Class 1']))

1/1 [==============================] - 0s 49ms/step
              precision    recall  f1-score   support

     Class 0       1.00      0.89      0.94         9
     Class 1       0.86      1.00      0.92         6

    accuracy                           0.93        15
   macro avg       0.93      0.94      0.93        15
weighted avg       0.94      0.93      0.93        15

